In [ ]:
import pandas as pd
from scipy.io import arff
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import GradientBoostingClassifier
import catboost as ctb
import numpy as np
import matplotlib.pyplot as plt
import warnings
from imblearn.over_sampling import SMOTE

# Ignorar avisos para manter a saída mais limpa
warnings.filterwarnings('ignore')

# --- 1. Análise e Preparação dos Dados ---

# Carregar o arquivo ARFF
try:
    data, meta = arff.loadarff('idosos_varRelevantes.arff')
except FileNotFoundError:
    print("Erro: O arquivo 'idosos_varRelevantes.arff' não foi encontrado.")
    exit()

# Converter para DataFrame e preparar os dados
df = pd.DataFrame(data)
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].str.decode('utf-8')
df.replace('?', np.nan, inplace=True)
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.dropna(inplace=True)
df['class'] = df['class'].astype(int)

# Separar variáveis preditoras (X) e alvo (y)
X = df.drop('class', axis=1)
y = df['class']

# Dividir em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# --- 2. Balanceamento dos Dados de Treino com SMOTE ---
print("--- Aplicando SMOTE aos dados de treino para balanceamento ---")
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
print(f"Distribuição de classes após SMOTE:\n{y_train_smote.value_counts()}\n")

# --- 3. Treinamento dos Novos Modelos ---

# --- Modelo 1: Logistic Regression com Cross-Validation ---
print("--- Treinando LogisticRegressionCV ---")
# O 'cv=5' faz validação cruzada para achar o melhor parâmetro de regularização
# 'scoring' focado em 'f1' para otimizar o balanço precisão-recall
log_reg_cv = LogisticRegressionCV(cv=5, random_state=42, scoring='f1', max_iter=1000)
log_reg_cv.fit(X_train_smote, y_train_smote)
print("Modelo treinado.\n")

# --- Modelo 2: Gradient Boosting do Scikit-learn ---
print("--- Treinando GradientBoostingClassifier ---")
gradient_boosting = GradientBoostingClassifier(n_estimators=150, learning_rate=0.1, random_state=42)
gradient_boosting.fit(X_train_smote, y_train_smote)
print("Modelo treinado.\n")

# --- Modelo 3: CatBoost (como referência de performance) ---
print("--- Treinando CatBoost ---")
catboost_smote = ctb.CatBoostClassifier(iterations=100, learning_rate=0.05, random_state=42, verbose=0)
catboost_smote.fit(X_train_smote, y_train_smote)
print("Modelo treinado.\n")

# --- 4. Avaliação Comparativa ---

# Fazer previsões no conjunto de teste (original, não balanceado)
y_pred_lr = log_reg_cv.predict(X_test)
y_pred_gb = gradient_boosting.predict(X_test)
y_pred_cat = catboost_smote.predict(X_test)

print("--- Resultados Finais: Logistic Regression ---")
print(classification_report(y_test, y_pred_lr))
print("--- Resultados Finais: Gradient Boosting ---")
print(classification_report(y_test, y_pred_gb))
print("--- Resultados Finais: CatBoost ---")
print(classification_report(y_test, y_pred_cat))

# --- 5. Geração do Gráfico Comparativo ---

# Extrair as métricas para o gráfico
report_lr_dict = classification_report(y_test, y_pred_lr, output_dict=True)
report_gb_dict = classification_report(y_test, y_pred_gb, output_dict=True)
report_cat_dict = classification_report(y_test, y_pred_cat, output_dict=True)

# Focaremos nas métricas da Classe 1 e na Acurácia geral
metrics_labels = ['Precision (Classe 1)', 'Recall (Classe 1)', 'F1-Score (Classe 1)', 'Accuracy']

scores = {
    'LogisticRegression': [report_lr_dict['1']['precision'], report_lr_dict['1']['recall'], report_lr_dict['1']['f1-score'], report_lr_dict['accuracy']],
    'GradientBoosting': [report_gb_dict['1']['precision'], report_gb_dict['1']['recall'], report_gb_dict['1']['f1-score'], report_gb_dict['accuracy']],
    'CatBoost': [report_cat_dict['1']['precision'], report_cat_dict['1']['recall'], report_cat_dict['1']['f1-score'], report_cat_dict['accuracy']],
}

x = np.arange(len(metrics_labels))
width = 0.25
fig, ax = plt.subplots(figsize=(16, 9))

# Criar as barras para cada modelo
rects1 = ax.bar(x - width, scores['LogisticRegression'], width, label='LogisticRegressionCV', color='cornflowerblue')
rects2 = ax.bar(x, scores['GradientBoosting'], width, label='GradientBoosting', color='seagreen')
rects3 = ax.bar(x + width, scores['CatBoost'], width, label='CatBoost (SMOTE)', color='lightcoral')

# Adicionar títulos e labels
ax.set_ylabel('Scores', fontsize=14)
ax.set_title('Comparação de Desempenho de Novos Modelos (Treinados com SMOTE)', fontsize=16)
ax.set_xticks(x)
ax.set_xticklabels(metrics_labels, fontsize=12)
ax.legend(fontsize=12)
ax.set_ylim(0, 1.1)

# Função para adicionar os valores exatos em cima das barras
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)
autolabel(rects3)

fig.tight_layout()

# Plotar gráfico
plt.show()